In [7]:
import os
import sys
import numpy as np
import pandas as pd
import scipy
from surprise import Reader, Dataset, SVD, evaluate, dump, accuracy, KNNBasic
from surprise.model_selection import GridSearchCV, cross_validate
from collections import defaultdict
import itertools

# Baseline algos to import
from surprise import KNNBasic

# Custom libraries
sys.path.append('../Util')
from joiner import get_ratings_pickle

In [2]:
# Set this to where you save and load all data
data_path = '../../goodbooks-10k/'

In [3]:
# Load ratings DF
ratings = get_ratings_pickle(data_path)

ratings_pickle existed as file...


In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userID', 'itemID', 'rating']], reader)

In [ ]:
trainset = data.build_full_trainset()

In [ ]:
# User KNN Basic

In [10]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
knn_basic = KNNBasic(sim_options=sim_options, verbose=True, k=2)

In [ ]:
# Run 5-fold cross-validation and print results
results = cross_validate(knn_basic, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)